# 거래를 위한 CNN - 1부: 특성 공학

시계열 데이터의 그리드형 구조를 활용하기 위해 단변량 및 다변량 시계열에 CNN 아키텍처를 사용할 수 있습니다. 후자의 경우, 서로 다른 색상 신호와 유사하게 서로 다른 시계열을 채널로 간주합니다.
대체 접근 방식은 시계열 알파 요소를 2차원 형식으로 변환하여 CNN의 로컬 패턴 감지 기능을 활용하는 것입니다. [세제르와 오즈바요글루(2018)](https://www.researchgate.net/publication/324802031_Algorithmic_Financial_Trading_with_Deep_Convolutional_Neural_Networks_Time_Series_to_Image_Conversion_Approach)은(는) 서로 다른 간격으로 15개의 기술 지표를 계산하고 계층적 클러스터링(13장, 비지도 학습을 통한 데이터 기반 위험 요소 및 자산 할당 참조)을 사용하여 2차원에서 서로 유사하게 동작하는 지표를 찾는 CNN-TA를 제안합니다. 그리드.
저자는 특정 날짜에 자산을 매수, 보유 또는 매도할지 예측하기 위해 이전에 사용한 CIFAR-10 예제와 유사한 CNN을 훈련합니다. 그들은 CNN 성과를 '매수 후 보유' 및 기타 모델과 비교하고 2007~2017년 기간 동안 다우 30개 주식과 가장 많이 거래된 9개 ETF의 일일 가격 시리즈를 사용하여 모든 대안보다 우수한 성과를 보인다는 사실을 발견했습니다.
*거래용 CNN* 섹션은 일일 미국 주식 가격 데이터를 사용하여 이 접근 방식을 실험하는 세 개의 노트북으로 구성됩니다. 그들은 시연한다1. 관련 재무 특성을 계산하는 방법2. 유사한 지표 집합을 이미지 형식으로 변환하고 유사성별로 클러스터링하는 방법3. 일일 수익률을 예측하고 결과 신호를 기반으로 간단한 롱-숏 전략을 평가하도록 CNN을 훈련하는 방법.

## 다양한 간격으로 기술 지표 생성우리는 먼저 2007년부터 2017년까지 5년 단위로 Quandl Wiki 데이터세트에서 달러 규모로 가장 많이 거래된 미국 주식 500개를 선택했습니다.
- 우리의 기능은 15개의 서로 다른 간격으로 계산한 다음 이를 15x15 그리드로 배열하는 15개의 기술 지표와 위험 요인으로 구성됩니다.- 각 지표에 대해 기간을 6에서 20까지 변경하여 15개의 개별 측정값을 얻습니다.

## 가져오기 및 설정

Python 3.7과 함께 `talib`을 설치하려면 [이것들](https://medium.com/@joelzhang/install-ta-lib-in-python-3-7-51219acacafb) 지침을 따르세요.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from talib import (RSI, BBANDS, MACD,
                   NATR, WILLR, WMA,
                   EMA, SMA, CCI, CMO,
                   MACD, PPO, ROC,
                   ADOSC, ADX, MOM)
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.regression.rolling import RollingOLS
import statsmodels.api as sm
import pandas_datareader.data as web
import pandas as pd
import numpy as np
from pathlib import Path
%matplotlib inline

In [3]:
DATA_STORE = '../data/assets.h5'

In [4]:
MONTH = 21
YEAR = 12 * MONTH

In [5]:
START = '2000-01-01'
END = '2017-12-31'

In [6]:
sns.set_style('whitegrid')
idx = pd.IndexSlice

In [7]:
T = [1, 5, 10, 21, 42, 63]

In [8]:
results_path = Path('results', 'cnn_for_trading')
if not results_path.exists():
    results_path.mkdir(parents=True)

## Quandl Wiki 주가 및 메타데이터 로드

In [9]:
adj_ohlcv = ['adj_open', 'adj_close', 'adj_low', 'adj_high', 'adj_volume']

In [10]:
with pd.HDFStore(DATA_STORE) as store:
    prices = (store['quandl/wiki/prices']
              .loc[idx[START:END, :], adj_ohlcv]
              .rename(columns=lambda x: x.replace('adj_', ''))
              .swaplevel()
              .sort_index()
             .dropna())
    metadata = (store['us_equities/stocks'].loc[:, ['marketcap', 'sector']])
ohlcv = prices.columns.tolist()

In [11]:
prices.volume /= 1e3
prices.index.names = ['symbol', 'date']
metadata.index.name = 'symbol'

## 롤링 유니버스: 가장 많이 거래되는 주식 500개 선택

In [12]:
dollar_vol = prices.close.mul(prices.volume).unstack('symbol').sort_index()

In [13]:
years = sorted(np.unique([d.year for d in prices.index.get_level_values('date').unique()]))

In [14]:
train_window = 5 # years
universe_size = 500

In [15]:
universe = []
for i, year in enumerate(years[5:], 5):
    start = str(years[i-5])
    end = str(years[i])
    most_traded = (dollar_vol.loc[start:end, :]
                   .dropna(thresh=1000, axis=1)
                   .median()
                   .nlargest(universe_size)
                   .index)
    universe.append(prices.loc[idx[most_traded, start:end], :])
universe = pd.concat(universe)

In [16]:
universe = universe.loc[~universe.index.duplicated()]

In [17]:
universe.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2530228 entries, ('BRK_A', Timestamp('2000-01-03 00:00:00')) to ('BLL', Timestamp('2017-12-29 00:00:00'))
Data columns (total 5 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   open    2530228 non-null  float64
 1   close   2530228 non-null  float64
 2   low     2530228 non-null  float64
 3   high    2530228 non-null  float64
 4   volume  2530228 non-null  float64
dtypes: float64(5)
memory usage: 106.4+ MB


In [18]:
universe.groupby('symbol').size().describe()

count     735.000000
mean     3442.487075
std      1145.365643
min      1043.000000
25%      2368.000000
50%      3792.000000
75%      4527.000000
max      4528.000000
dtype: float64

In [19]:
universe.to_hdf('data.h5', 'universe')

## 기술 지표 요소 생성

In [20]:
T = list(range(6, 21))

### 상대 강도 지수

In [21]:
for t in T:
    universe[f'{t:02}_RSI'] = universe.groupby(level='symbol').close.apply(RSI, timeperiod=t)

### 윌리엄스%R

In [22]:
for t in T:
    universe[f'{t:02}_WILLR'] = (universe.groupby(level='symbol', group_keys=False)
     .apply(lambda x: WILLR(x.high, x.low, x.close, timeperiod=t)))

### 볼린저 밴드 계산

In [23]:
def compute_bb(close, timeperiod):
    high, mid, low = BBANDS(close, timeperiod=timeperiod)
    return pd.DataFrame({f'{timeperiod:02}_BBH': high, f'{timeperiod:02}_BBL': low}, index=close.index)

In [24]:
for t in T:
    bbh, bbl = f'{t:02}_BBH', f'{t:02}_BBL'
    universe = (universe.join(
        universe.groupby(level='symbol').close.apply(compute_bb,
                                                     timeperiod=t)))
    universe[bbh] = universe[bbh].sub(universe.close).div(universe[bbh]).apply(np.log1p)
    universe[bbl] = universe.close.sub(universe[bbl]).div(universe.close).apply(np.log1p)

### 정규화된 평균 실제 범위

In [25]:
for t in T:
    universe[f'{t:02}_NATR'] = universe.groupby(level='symbol', 
                                group_keys=False).apply(lambda x: 
                                                        NATR(x.high, x.low, x.close, timeperiod=t))

### 백분율 가격 발진기

In [26]:
for t in T:
    universe[f'{t:02}_PPO'] = universe.groupby(level='symbol').close.apply(PPO, fastperiod=t, matype=1)

### 이동 평균 수렴/발산

In [27]:
def compute_macd(close, signalperiod):
    macd = MACD(close, signalperiod=signalperiod)[0]
    return (macd - np.mean(macd))/np.std(macd)

In [28]:
for t in T:
    universe[f'{t:02}_MACD'] = (universe
                  .groupby('symbol', group_keys=False)
                  .close
                  .apply(compute_macd, signalperiod=t))

### 기세

In [29]:
for t in T:
    universe[f'{t:02}_MOM'] = universe.groupby(level='symbol').close.apply(MOM, timeperiod=t)

### 가중이동평균

In [30]:
for t in T:
    universe[f'{t:02}_WMA'] = universe.groupby(level='symbol').close.apply(WMA, timeperiod=t)

### 지수 이동 평균

In [31]:
for t in T:
    universe[f'{t:02}_EMA'] = universe.groupby(level='symbol').close.apply(EMA, timeperiod=t)

### 상품 채널 지수

In [32]:
for t in T:    
    universe[f'{t:02}_CCI'] = (universe.groupby(level='symbol', group_keys=False)
     .apply(lambda x: CCI(x.high, x.low, x.close, timeperiod=t)))

### Chande 모멘텀 발진기

In [33]:
for t in T:
    universe[f'{t:02}_CMO'] = universe.groupby(level='symbol').close.apply(CMO, timeperiod=t)

### 변화율

변화율은 일정 기간 동안의 가격 변화 속도를 나타내는 기술 지표입니다.

In [34]:
for t in T:
    universe[f'{t:02}_ROC'] = universe.groupby(level='symbol').close.apply(ROC, timeperiod=t)

### Chaikin A/D 발진기

In [35]:
for t in T:
    universe[f'{t:02}_ADOSC'] = (universe.groupby(level='symbol', group_keys=False)
     .apply(lambda x: ADOSC(x.high, x.low, x.close, x.volume, fastperiod=t-3, slowperiod=4+t)))

### 평균 방향 이동 지수

In [36]:
for t in T:
    universe[f'{t:02}_ADX'] = universe.groupby(level='symbol', 
                                group_keys=False).apply(lambda x: 
                                                        ADX(x.high, x.low, x.close, timeperiod=t))

In [37]:
universe.drop(ohlcv, axis=1).to_hdf('data.h5', 'features')

## 과거 수익률 계산

### 역사적 수익률

In [38]:
by_sym = universe.groupby(level='symbol').close
for t in [1,5]:
    universe[f'r{t:02}'] = by_sym.pct_change(t)

### 이상값 제거

In [39]:
universe[[f'r{t:02}' for t in [1, 5]]].describe()

,r01,r05
count,2.529493e+06,2.526553e+06
mean,6.710840e-04,3.293540e-03
std,2.875355e-02,6.344951e-02
min,-9.718670e-01,-9.795396e-01
25%,-1.034141e-02,-2.246575e-02
50%,3.236246e-04,2.921130e-03
75%,1.122661e-02,2.811951e-02
max,1.216425e+01,1.252657e+01


In [40]:
outliers = universe[universe.r01>1].index.get_level_values('symbol').unique()
len(outliers)

11

In [41]:
universe = universe.drop(outliers, level='symbol')

### 과거 수익률 분위수

In [42]:
for t in [1, 5]:
    universe[f'r{t:02}dec'] = (universe[f'r{t:02}'].groupby(level='date')
             .apply(lambda x: pd.qcut(x, q=10, labels=False, duplicates='drop')))

## 롤링 팩터 베타

또한 5가지 Fama-French 위험 요소를 사용합니다(Fama and French, 2015; 4장 금융 특성 엔지니어링 – 알파 요소 조사 방법 참조). 이는 주식 수익률에 영향을 미치는 것으로 지속적으로 입증된 요인에 대한 주식 수익률의 민감도를 반영합니다.
우리는 기본 동인을 반영하도록 설계된 포트폴리오 수익률에 대한 주식 일일 수익률의 롤링 OLS 회귀 계수를 계산하여 이러한 요소를 포착합니다.- **주식 리스크 프리미엄**: 미국 주식의 가치 가중 수익률에서 미국 1개월 수익률을 차감- **재무부 청구율**- **규모(SMB)**: 소형(시가총액 기준)으로 분류된 주식에서 대형주를 뺀 수익률- **가치(HML)**: 장부가치가 높은 주식에서 가치가 낮은 주식을 뺀 수익률- **투자(CMA)**: 보수적 투자 지출을 한 기업의 수익률 차이에서 공격적 지출을 한 기업을 뺀 값입니다.- **수익성(RMW)**: 마찬가지로 견고한 수익성을 가진 주식의 수익률 차이에서 약한 지표를 가진 주식의 수익률 차이를 뺍니다.


In [43]:
factor_data = (web.DataReader('F-F_Research_Data_5_Factors_2x3_daily', 'famafrench', 
                              start=START)[0].rename(columns={'Mkt-RF': 'Market'}))
factor_data.index.names = ['date']

In [44]:
factor_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5284 entries, 2000-01-03 to 2020-12-31
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Market  5284 non-null   float64
 1   SMB     5284 non-null   float64
 2   HML     5284 non-null   float64
 3   RMW     5284 non-null   float64
 4   CMA     5284 non-null   float64
 5   RF      5284 non-null   float64
dtypes: float64(6)
memory usage: 289.0 KB


In [45]:
windows = list(range(15, 90, 5))
len(windows)

15

다음으로, `statsmodels`' `RollingOLS()`을 적용하여 15일에서 90일까지 다양한 길이의 기간 동안 회귀를 실행합니다. `.fit()` 메서드에 `params_only` 매개변수를 설정하여 계산 속도를 높이고 적합한 Factor_model의 `.params` 속성을 사용하여 계수를 캡처합니다.

In [46]:
t = 1
ret = f'r{t:02}'
factors = ['Market', 'SMB', 'HML', 'RMW', 'CMA']
windows = list(range(15, 90, 5))
for window in windows:
    print(window)
    betas = []
    for symbol, data in universe.groupby(level='symbol'):
        model_data = data[[ret]].merge(factor_data, on='date').dropna()
        model_data[ret] -= model_data.RF

        rolling_ols = RollingOLS(endog=model_data[ret], 
                                 exog=sm.add_constant(model_data[factors]), window=window)
        factor_model = rolling_ols.fit(params_only=True).params.drop('const', axis=1)
        result = factor_model.assign(symbol=symbol).set_index('symbol', append=True)
        betas.append(result)
    betas = pd.concat(betas).rename(columns=lambda x: f'{window:02}_{x}')
    universe = universe.join(betas)

15
20
25
30
35
40
45
50
55
60
65
70
75
80
85


## 전방 수익 계산

In [47]:
for t in [1, 5]:
    universe[f'r{t:02}_fwd'] = universe.groupby(level='symbol')[f'r{t:02}'].shift(-t)
    universe[f'r{t:02}dec_fwd'] = universe.groupby(level='symbol')[f'r{t:02}dec'].shift(-t)

## 모델 데이터 저장

In [48]:
universe = universe.drop(ohlcv, axis=1)

In [49]:
universe.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2499265 entries, ('BRK_A', Timestamp('2000-01-03 00:00:00')) to ('BLL', Timestamp('2017-12-29 00:00:00'))
Columns: 308 entries, 06_RSI to r05dec_fwd
dtypes: float64(308)
memory usage: 5.7+ GB


In [50]:
drop_cols = ['r01', 'r01dec', 'r05',  'r05dec']

In [51]:
outcomes = universe.filter(like='_fwd').columns

In [52]:
universe = universe.sort_index()
with pd.HDFStore('data.h5') as store:
    store.put('features', universe.drop(drop_cols, axis=1).drop(outcomes, axis=1).loc[idx[:, '2001':], :])
    store.put('targets', universe.loc[idx[:, '2001':], outcomes])